In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databrickseteproject167.dfs.core.windows.net/products")
df = df.drop("_rescued_data")
df.display()

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databrickseteproject167_catalog.bronze.discount_func(principal_amount double)
RETURNS double
RETURN principal_amount * 0.90

In [0]:
df.withColumn("discount_price", expr("databrickseteproject167_catalog.bronze.discount_func(price)")).display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databrickseteproject167_catalog.bronze.upper_func(val STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
RETURN val.upper()
$$

In [0]:
%sql
CREATE OR REPLACE FUNCTION databrickseteproject167_catalog.bronze.lower_func(val STRING)
RETURNS STRING
RETURN lower(val)

In [0]:
df.display()

In [0]:
df.write.format("delta").mode("overwrite").option("path","abfss://silver@databrickseteproject167.dfs.core.windows.net/products").save()

In [0]:
display(dbutils.fs.ls("abfss://silver@databrickseteproject167.dfs.core.windows.net/products"))

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databrickseteproject167_catalog.silver.products
USING DELTA
LOCATION 'abfss://silver@databrickseteproject167.dfs.core.windows.net/products'